In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [19]:
data = pd.read_csv("/content/compressed_data.csv")
data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,1,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,1,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,1,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,1,0,3


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Soil_Type1                          581012 non-null 

In [21]:
print(data.isnull().sum())

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0
Soil_Type12                           0
Soil_Type13                           0
Soil_Type14                           0


In [24]:
# Pisahkan fitur dan target
X = data.drop(columns=["Cover_Type"]).values
y = (data["Cover_Type"] - 1).values  # Ubah kelas dari 1-7 menjadi 0-6 untuk kompatibilitas

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [25]:
# Standarisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
# TensorFlow Model
model_tf = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_tf.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

y_pred_tf = model_tf.predict(X_test)
y_pred_classes_tf = np.argmax(y_pred_tf, axis=1)

accuracy_tf = accuracy_score(y_test, y_pred_classes_tf)
precision_tf = precision_score(y_test, y_pred_classes_tf, average='weighted')
recall_tf = recall_score(y_test, y_pred_classes_tf, average='weighted')
f1_tf = f1_score(y_test, y_pred_classes_tf, average='weighted')
auc_roc_tf = roc_auc_score(pd.get_dummies(y_test), y_pred_tf, multi_class='ovr')

print(f"TensorFlow - Accuracy: {accuracy_tf:.4f}")
print(f"TensorFlow - Precision: {precision_tf:.4f}")
print(f"TensorFlow - Recall: {recall_tf:.4f}")
print(f"TensorFlow - F1 Score: {f1_tf:.4f}")
print(f"TensorFlow - AUC-ROC: {auc_roc_tf:.4f}")

Epoch 1/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 55s 4ms/step - accuracy: 0.7069 - loss: 0.6958 - val_accuracy: 0.7819 - val_loss: 0.5075
Epoch 2/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 76s 3ms/step - accuracy: 0.7619 - loss: 0.5512 - val_accuracy: 0.7975 - val_loss: 0.4685
Epoch 3/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.7747 - loss: 0.5209 - val_accuracy: 0.8037 - val_loss: 0.4525
Epoch 4/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.7821 - loss: 0.5075 - val_accuracy: 0.8172 - val_loss: 0.4317
Epoch 5/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 0.7880 - loss: 0.4952 - val_accuracy: 0.8238 - val_loss: 0.4210
Epoch 6/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - accuracy: 0.7900 - loss: 0.4886 - val_accuracy: 0.8260 - val_loss: 0.4160
Epoch 7/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 87s 3ms/step - accuracy: 0.7944 - loss: 0.4803 - val_accuracy: 0.8284 - val_loss: 0.4107
Epoch 8/20
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 45s 3ms/step - accuracy: 